In [57]:
!pip install mlxtend > /dev/null 

In [158]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from collections import OrderedDict
import pandas as pd

# Практическое задание 3
## Вариант 4

## Задание 1 

Дан набор данных:

| tid              | itemset |
| ---------------- | ------- |
| *t<sub>1</sub>*  | *CFG*    |
| *t<sub>2</sub>*  | *AF*    |
| *t<sub>3</sub>*  | *BDG*    |
| *t<sub>4</sub>*  | *BCEF*    |
| *t<sub>5</sub>*  | *ACDEF*    |
| *t<sub>6</sub>*  | *ADF*    |
| *t<sub>7</sub>*  | *AB*    |
| *t<sub>8</sub>*  | *AB*    |
| *t<sub>9</sub>*  | *BD*   |
| *t<sub>10</sub>* | *ACE*   |

### Алгоритм Apriori

Минимальный уровень поддержки равен **4 / 10** 

1.1. *Самостоятельно* реализуйте алгоритм **Apriori** и продемонстрируйте, как алгоритм перебирает предложенный набор данных. Выведите результат работы алгоритма

1.2. Воспользуйтесь алгоритмом **Apriori** из библиотеки **MLxtend** и выведите результат его работы. Сравните результаты собственной и библиотечной реализации

In [217]:
%%time
print("1.1")
# В работе использовал генераторы и множества. Преимущество данных структур в питоне в том, что они используют хэш.
# То есть за счет хэша производится не наивный поиск по строке, а по поиск по хэшу, что быстрее, чем матрица смежности n*m,
# особенно, когда асоциации небольшие по длине

SUPPORT_LEVEL = 4
count = 1
### Мой сет по заданию, пр. 3, вариант 4
items=["CFG","AF","BDG","BCEF","ACDEF","ADF","AB", "AB", "BD", "ACE"]

#### Тестовые сеты
#items=["CFG","AF","BDG","BCEF","ACDEF","AСDF","AB", "AB", "BD", "ACE"]
#items = ["ABCFG", "ABCF", "ABDG", "BCEF", "ACDEF", "ACDF", "ACB", "ACB", "BD", "ACE"]


def_dict = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0}
mass = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}



for sub in def_dict.keys():
    # из подстроки строится множество, оно разбивается на отдельные буквы
    # с его помощью можно найти все сочетание букв, например AC в строке ABC, как 'A','C', при этом true будет только, если
    # всё множесто входит в лексему (то есть и 'A', и 'С')
    res = len([k for k in items if set(sub) <= set(k)]) # счетчик через генератор на основе множества
    def_dict[sub] = res
print("Демонстрация перебора")
print("Было, {} итерация: ".format(count))
for k, v in def_dict.items():
    print(k,"|", v,"|",end='')
    if (v < SUPPORT_LEVEL):
        print("false")
    else:
        print("true")
res_dict = {}
def_dict = {k: v for k, v in def_dict.items() if v >= SUPPORT_LEVEL} # проверка на минимальный уровень поддержки

new_dict = def_dict
print("Стало, {} итерация: ".format(count))
for k, v in new_dict.items():
    print(k,"|", v,"|",end='')
    if (v < SUPPORT_LEVEL):
        print("false")
    else:
        print("true")
print()
res_dict.update(new_dict)
# print(res_dict)

check_old = list(def_dict.keys())

# для случая комбинаций лексем (n мерный), для таких лексем можно проверить путем изменения константы {SUPPORT_LEVEL}
while (new_dict):
    check_new = list(new_dict.keys())
    for i in check_new:
        # i всегда из нескольких букв, упорядоченных по алфавиту, интересна последняя буква, так как ее числовой эквивалент
        # определяет комбинации
        for j in check_old:# j всегда из одной буквы (старого массива, полученного при инициализации выше, соот. словарю)
            if mass[i[-1]] < mass[j]:
                new_dict[i + j] = 0
    count +=  1
    new_dict = {k: v for k, v in new_dict.items() if len(k) == count}
    for sub in new_dict.keys():
        res = len([k for k in items if set(sub) <= set(k)])
        new_dict[sub] = res
    print("Было, {} итерация: ".format(count))
    for k, v in new_dict.items():
        print(k,"|", v,"|",end='')
        if (v < SUPPORT_LEVEL):
            print("false")
        else:
            print("true")
    new_dict = {k: v for k, v in new_dict.items() if v >= SUPPORT_LEVEL}
    print("Стало, {} итерация: ".format(count))
    for k, v in new_dict.items():
        print(k,"|", v,"|",end='')
        if (v < SUPPORT_LEVEL):
            print("false")
        else:
            print("true")
    print()
    res_dict.update(new_dict)
    if (count > 999998):
        # сюда попадать не должен, но на всякий случай поставлена заглушка
        print("Юпитер не бань))")
        exit()
print("-------------------------------------\n")
print("Результат: ")
for k, v in res_dict.items():
    print(k, v)
print()

1.1
Демонстрация перебора
Было, 1 итерация: 
A | 6 |true
B | 5 |true
C | 4 |true
D | 4 |true
E | 3 |false
F | 5 |true
G | 2 |false
Стало, 1 итерация: 
A | 6 |true
B | 5 |true
C | 4 |true
D | 4 |true
F | 5 |true

Было, 2 итерация: 
AB | 2 |false
AC | 2 |false
AD | 2 |false
AF | 3 |false
BC | 1 |false
BD | 2 |false
BF | 1 |false
CD | 1 |false
CF | 3 |false
DF | 2 |false
Стало, 2 итерация: 

-------------------------------------

Результат: 
A 6
B 5
C 4
D 4
F 5

CPU times: user 560 µs, sys: 0 ns, total: 560 µs
Wall time: 532 µs


In [216]:
%%time
print("1.2")
### Мой сет по заданию, пр. 3, вариант 4
items=["CFG","AF","BDG","BCEF","ACDEF","ADF","AB", "AB", "BD", "ACE"]


print("Результаты полученные с помощью собственноручно написаннного алгоритма и библиотечного")
print("полностью сходятся\n")

te = TransactionEncoder()
te_ary = te.fit(items).transform(items)
df = pd.DataFrame(te_ary, columns=te.columns_)
apriori(df, min_support=0.4, use_colnames=True)

1.2
Результаты полученные с помощью собственноручно написаннного алгоритма и библиотечного
полностью сходятся

CPU times: user 2.49 ms, sys: 1.17 ms, total: 3.66 ms
Wall time: 3.29 ms


,support,itemsets
0,0.6,(A)
1,0.5,(B)
2,0.4,(C)
3,0.4,(D)
4,0.5,(F)


# Примечание = ДОП инфа по Apriori)))

По проверки времени __%%time__ моя реализация алгоритма работает примерно __в 7 раз быстрее__ (500 микросекунд против 3,5 ms  = 3500 микросекунд), чем библиотечная. На тестовых сетах зависимость "в 7 раз"  +- также сохраняется. С учетом print 

PS: по крайней мере точно быстрее на небольших сетах, вероятно работа с pandas будет выиграшна в Big Data 


### Алгоритм FPGrowth

Минимальный уровень поддержки равен **3 / 10**

1.3.  *Самостоятельно* реализуйте алгоритм **FPGrowth** и продемонстрируйте, как алгоритм перебирает предложенный набор данных. Выведите результат работы алгоритма

1.4. Воспользуйтесь алгоритмом **FPGrowth** из библиотеки **MLxtend** и выведите результат его работы. Сравните результаты собственной и библиотечной реализации

In [215]:
print("1.3")
import copy
SUPPORT_LEVEL = 3
next_step = []
BIG_END = []
LETTER_REC = ''
in_count = 0
class treeNode:# класс дерева
    def __init__(self, name_val, counter, parent_node):
        self.name = name_val # имя
        self.count = counter # счетчик
        self.nodeLink = None 
        self.parent = parent_node #родитель
        self.mark = None #опознователь
        self.children = {}

    def inc(self, counter):
        self.count += counter

    @property
    def give_name(self):
        return self.name
    
    def disp(self, ind=1): # показ дерева
        if self is not None:
            print('-' * ind, self.name, 'count:', self.count)
            for child in self.children.values():
                child.disp(ind + 1)
            
    def disp_plus(self, letter, ind=1): # проход для условного дерева с установлением counter с рекурсией
            if self is not None:
                if self.name == letter:
                    back = go_back_to_root(self)
                for child in self.children.values():
                    child.disp_plus(letter, ind + 1)
    def dsp_res(self, letter, ind=1):# проход для условного дерева с рекурсией для вывода
            global next_step
            if ind == 1:
                next_step = []
            if self is not None:
                if self.name == letter:

                    back2 = go_back_to_root_2(self)
                    next_step.append(back2)

                for child in self.children.values():
                    child.dsp_res(letter, ind + 1)
                    
def go_back_to_root(self): # к корню с counter
    some_list=[]
    letterCount = 0
    if (self.name != 'root'):
        letterCount = self.count
        self = self.parent
    while(self.name != 'root'):
        if self.mark is None: 
            self.count = letterCount
            self.mark = 1;
        else:
            self.count += letterCount
        some_list.append(self.name)
        self = self.parent
    some_list.reverse()
    return some_list
def go_back_to_root_2(self):# к корню для вывода
    some_list=[]
    if (self.name != 'root'):
        letterCount = self.count
        if self.mark != None:
            self.mark = None
        self = self.parent
    while(self.name != 'root'):
        if self.mark != None:
            self.mark = None
            for i in range(0,self.count):
                some_list.append(self.name)            
        self = self.parent
    some_list.reverse()
    return some_list


def create_tree(dataSet, minSup, header_table):# создание дерева
    freq_item_set = set(header_table.keys())

    for k in header_table:
        header_table[k] = [header_table[k], None]
    ret_tree = treeNode('root', 1, None) # корневой нод
    for tran_set, count in dataSet.items():
        loc_dict = {}
        for item in tran_set:
            if item in freq_item_set:
                loc_dict[item] = header_table[item][0]
        if len(loc_dict) > 0:
            ordered_items = [v[0] for v in sorted(loc_dict.items(), key=lambda p: (p[1], p[0]), reverse=True)]# сортировка
            update_tree(ordered_items, ret_tree, header_table, count)
    return ret_tree, header_table


def fp_tree_prepare_date(items):# подготовка данных
    def_dict = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0}# подсчет header_table - головной таблицы для словаря
    for sub in def_dict.keys():
        res = len([k for k in items if set(sub) <= set(k)])  # счетчик через генератор на основе множества
        def_dict[sub] = res
    res_dict = {}
    def_dict = {k: v for k, v in def_dict.items() if v >= SUPPORT_LEVEL}  # проверка на минимальный уровень поддержки
    return def_dict


def update_tree(items, in_tree, header_table, count): # обновление дерева при добавлении
    if items[0] in in_tree.children:
        in_tree.children[items[0]].inc(count)
    else:
        in_tree.children[items[0]] = treeNode(items[0], count, in_tree)
        if header_table[items[0]][1] == None:  
            header_table[items[0]][1] = in_tree.children[items[0]]
        else:
            update_node(header_table[items[0]][1], in_tree.children[items[0]])
    if len(items) > 1:
        update_tree(items[1:], in_tree.children[items[0]], header_table, count)


def create_init_set(dataSet): # заморозка
    ret_dict = {}
    for trans in dataSet:
        fset = frozenset(trans)
        ret_dict.setdefault(fset, 0)
        ret_dict[fset] += 1

    return ret_dict



def update_node(node_to_test, target_node): # для обновление дерева обновление нода
    while (node_to_test.nodeLink != None):
        node_to_test = node_to_test.nodeLink
    node_to_test.nodeLink = target_node

def search_tokens(myFPTree, letter, token): # поиск токенов через n мерную рекурсию
    header_table = {}
    if token == None:
        myFPTreeDop = copy.deepcopy(myFPTree)
        
        myFPTreeDop.disp_plus(letter) 
        myFPTreeDop.dsp_res(letter)
        myFPTreeDopEt = copy.deepcopy(myFPTreeDop)# копия для обработки
        
        for trans in next_step:
             for item in trans:
                header_table[item] = header_table.get(item, 0) + 1
        header_table = {k: v for k, v in header_table.items() if v >= SUPPORT_LEVEL}
        if len(header_table) > 1:
            for k,v in header_table.items():
                token = letter + k
                print(token, v)
                search_tokens(myFPTreeDop, k, token)
                myFPTreeDop = copy.deepcopy(myFPTreeDopEt)
            
        else:
            for k,v in header_table.items():
                print(letter+k,v)
    else:
        #зона рекурсии
        global in_count
        
        
        # myFPTree.disp()
        myFPTree.disp_plus(letter)
        myFPTree.dsp_res(letter)
        # myFPTree.disp()
        
        for trans in next_step:
             for item in trans:
                header_table[item] = header_table.get(item, 0) + 1

        header_table = {k: v for k, v in header_table.items() if v >= SUPPORT_LEVEL}
        if len(header_table) > 1:
            for k,v in header_table.items():
                token = token + k 
                print(token, v)
                
                in_count += 1 
                search_tokens(myFPTree, k, token)
                token = token[:-1]
        else:
            for k,v in header_table.items():
                print(token+k,v)
if __name__ == '__main__':
    items = ["CFG", "AF", "BDG", "BCEF", "ACDEF", "ADF", "AB", "AB", "BD", "ACE"]
    dop_table = fp_tree_prepare_date(items)
    dop_table={k: v for k, v in sorted(dop_table.items(), key=lambda item: item[1])}

    
    dataset = [list(i) for i in items]# для последующей заморозки сета


    str_data = create_init_set(dataset)

    if (str_data is not None):
        header_table = fp_tree_prepare_date(items)
        myFPTree, myheader = create_tree(str_data, SUPPORT_LEVEL, header_table)
    print("Структура FPTree:\n")
    myFPTree.disp()
    print()
    print("----------------")
    print("Результат:")
    recursion = False
    for k,v in dop_table.items():
        print(k, v)
        search_tokens(myFPTree, k, None)
        global next_step
        next_step = []

1.3
Структура FPTree:

- root count: 1
-- F count: 2
--- C count: 1
--- B count: 1
---- C count: 1
----- E count: 1
-- A count: 6
--- F count: 3
---- D count: 2
----- C count: 1
------ E count: 1
--- B count: 2
--- C count: 1
---- E count: 1
-- B count: 2
--- D count: 2

----------------
Результат:
E 3
EC 3
C 4
CF 3
D 4
B 5
F 5
FA 3
A 6


In [214]:
print("1.4")

### Мой сет по заданию, пр. 3, вариант 4
items=["CFG","AF","BDG","BCEF","ACDEF","ADF","AB", "AB", "BD", "ACE"]

print("Результаты полученные с помощью собственноручно написаннного алгоритма и библиотечного")
print("полностью сходятся \n")

te = TransactionEncoder()
te_ary = te.fit(items).transform(items)
df = pd.DataFrame(te_ary, columns=te.columns_)
fpgrowth(df, min_support=0.3, use_colnames=True)

1.4
Результаты полученные с помощью собственноручно написаннного алгоритма и библиотечного
полностью сходятся 



,support,itemsets
0,0.5,(F)
1,0.4,(C)
2,0.6,(A)
3,0.5,(B)
4,0.4,(D)
5,0.3,(E)
6,0.3,"(A, F)"
7,0.3,"(F, C)"
8,0.3,"(C, E)"


## Задание 2

На рисунке представлена классификация различных продуктов. Каждый лист дерева это конкретный продукт, внутренний узел дерева представляет категорию продукта более верхнего уровня.

![a](n3_4_1.png)

Был получен следующий набор данных:

| tid  | itemset |
| ---- | ------- |
| *t<sub>1</sub>*  | *BDH*    |
| *t<sub>2</sub>*  | *DGH*    |
| *t<sub>3</sub>*  | *ABDEG*    |
| *t<sub>4</sub>*  | *ADEIJ*    |
| *t<sub>5</sub>*  | *ABEI*    |
| *t<sub>6</sub>*  | *AEFGIJ*    |
| *t<sub>7</sub>*  | *BCEGH*    |
| *t<sub>8</sub>*  | *BCDFGJ*    |
| *t<sub>9</sub>*  | *ABDEHIJ*   |
| *t<sub>10</sub>* | *BFIJ*   |


Выполните следующие задание:

2.1. Каков размер области поиска наборов элементов, если ограничиваться только наборами, состоящими из простых элементов?

2.2. Минимальный уровень поддержки равен **7 / 10**. Найдите все часто встречающиеся наборы элементов, состоящие только из элементов высокого уровня в таксономии. Имейте в виду, что если в транзакции появляется простой элемент, предполагается, что все его предки высокого уровня также присутствуют в транзакции. При выполнении данного пункта воспользуйтесь собственной реализацией **Apriori**.

In [ ]:
print("2.1")
print("Область из 10 элементов")

In [213]:
print("2.2")

SUPPORT_LEVEL = 7
count = 1

dp_items = ["BDH", "DGH", "ABDEG", "ADEIJ", "ABEI","AEFGIJ","BCEGH","BCDFGJ","ABDEHIJ","BFIJ"]
alphach = ['A','B','N','J']

dop_items = [j if j in alphach else 'NN' for i in dp_items for j in list(i) ]#

items=[]
h_count = 0
p_count = 0
in_str = ""
j = 1
for i in range(0, len(dop_items)):
    if h_count < len(dp_items[p_count]):
        in_str += dop_items[i] 
        h_count += 1
        if (i==(len(dop_items))-1):
            items.append(in_str)
    else:
        items.append(in_str)
        in_str = ""
        h_count = 0
        h_count += 1
        in_str += dop_items[i] 
        p_count += 1

def_dict = {'A': 0, 'B': 0, 'N': 0, 'J': 0}
mass  = {'A': 0, 'B': 1, 'N': 2, 'J': 3}

for sub in def_dict.keys():
    # из подстроки строится множество, оно разбивается на отдельные буквы
    # с его помощью можно найти все сочетание букв, например AC в строке ABC, как 'A','C', при этом true будет только, если
    # всё множесто входит в лексему (то есть и 'A', и 'С')
    res = len([k for k in items if set(sub) <= set(k)]) # счетчик через генератор на основе множества
    def_dict[sub] = res
print("Демонстрация перебора")
print("Было, {} итерация: ".format(count))
for k, v in def_dict.items():
    print(k,"|", v,"|",end='')
    if (v < SUPPORT_LEVEL):
        print("false")
    else:
        print("true")
res_dict = {}
def_dict = {k: v for k, v in def_dict.items() if v >= SUPPORT_LEVEL} # проверка на минимальный уровень поддержки

new_dict = def_dict
print("Стало, {} итерация: ".format(count))
for k, v in new_dict.items():
    print(k,"|", v,"|",end='')
    if (v < SUPPORT_LEVEL):
        print("false")
    else:
        print("true")
print()
res_dict.update(new_dict)
# print(res_dict)

check_old = list(def_dict.keys())

# для случая комбинаций лексем (n мерный), для таких лексем можно проверить путем изменения константы {SUPPORT_LEVEL}
while (new_dict):
    check_new = list(new_dict.keys())
    for i in check_new:
        # i всегда из нескольких букв, упорядоченных по алфавиту, интересна последняя буква, так как ее числовой эквивалент
        # определяет комбинации
        for j in check_old:# j всегда из одной буквы (старого массива, полученного при инициализации выше, соот. словарю)
            if mass[i[-1]] < mass[j]:
                new_dict[i + j] = 0
    count +=  1
    new_dict = {k: v for k, v in new_dict.items() if len(k) == count}
    for sub in new_dict.keys():
        res = len([k for k in items if set(sub) <= set(k)])
        new_dict[sub] = res
    print("Было, {} итерация: ".format(count))
    for k, v in new_dict.items():
        print(k,"|", v,"|",end='')
        if (v < SUPPORT_LEVEL):
            print("false")
        else:
            print("true")
    new_dict = {k: v for k, v in new_dict.items() if v >= SUPPORT_LEVEL}
    print("Стало, {} итерация: ".format(count))
    for k, v in new_dict.items():
        print(k,"|", v,"|",end='')
        if (v < SUPPORT_LEVEL):
            print("false")
        else:
            print("true")
    print()
    res_dict.update(new_dict)
    if (count > 999998):
        # сюда попадать не должен, но на всякий случай поставлена заглушка
        print("Юпитер не бань))")
        exit()
print("-------------------------------------\n")
print("Результат: ")
for k, v in res_dict.items():
    print(k, v)
print()

2.2
Демонстрация перебора
Было, 1 итерация: 
A | 5 |false
B | 7 |true
N | 10 |true
J | 5 |false
Стало, 1 итерация: 
B | 7 |true
N | 10 |true

Было, 2 итерация: 
BN | 7 |true
Стало, 2 итерация: 
BN | 7 |true

Было, 3 итерация: 
Стало, 3 итерация: 

-------------------------------------

Результат: 
B 7
N 10
BN 7



## Для справки

1. Машинное обучение. Учебное пособие. Темы: 
   - Поиск наборов объектов. Поддержка и часто встречающиеся наборы элементов. Алгоритмы майнинга набора данных
   - Поуровневый подход: алгоритм Aprior. Алгоритм 4.2
   - Подход дерева частоты образцов: алгоритмов FPGrowth. Алгоритм 4.5
2. Алгоритм FPGrowth (pdf файл) - описание и пример работы
3. [Apriori Algorithm In Data Mining: Implementation With Examples](https://www.softwaretestinghelp.com/apriori-algorithm/)
4. [Frequent Pattern (FP) Growth Algorithm In Data Mining](https://www.softwaretestinghelp.com/fp-growth-algorithm-data-mining/)